TODO

1. remove dropout from exported programs
2. вероятно нужно сделать модули FullEncoder 
3. проверить, игнорируются поля модели, которые не используются в forward (например, есть ли разница между текущей инмплементацией `Decode` и имплементацией, где вся модель хранится как удинственное поле Decode) 

In [57]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
from typing import Dict, Union, Tuple
import os
import array

import torch
from torch import Tensor
from torch.export import export, ExportedProgram, Dim
from executorch.exir import EdgeProgramManager, to_edge, to_edge_transform_and_lower
from executorch.exir.backend.backend_api import LoweredBackendModule, to_backend
from executorch.backends.xnnpack.partition.xnnpack_partitioner import XnnpackPartitioner

from model import MODEL_GETTERS_DICT
from feature_extractors import get_val_transform
from ns_tokenizers import CharLevelTokenizerv2, KeyboardTokenizerv1
from word_generators_v2 import BeamGenerator

In [59]:
def remove_prefix(s: str, prefix: str) -> str:
    if s.startswith(prefix):
        s = s[len(prefix):]
    return s


def get_state_dict_from_checkpoint(ckpt: dict) -> Dict[str, torch.Tensor]:
    return {remove_prefix(k, 'model.'): v for k, v in ckpt['state_dict'].items()}

In [60]:
# COMMAND LINE ARGUMENTS EMULATION

MODEL_NAME = 'v3_nearest_and_traj_transformer_bigger'
CHECKPOINT_ROOT_PATH = '../../../checkpoints_for_executorch/my_nearest_features/'
CHECKPOINT_PATH = os.path.join(CHECKPOINT_ROOT_PATH, 'v3_nearest_and_traj_transformer_bigger-default--epoch=73-val_loss=0.444-val_word_level_accuracy=0.872.ckpt')
TRANSFORM_NAME =  "traj_feats_and_nearest_key"

DATA_ROOT = '../data/data_separated_grid'

GRIDNAME_TO_GRID_PATH = os.path.join(DATA_ROOT, "gridname_to_grid.json")
voc_path=os.path.join(DATA_ROOT, "voc.txt")
char_tokenizer = CharLevelTokenizerv2(voc_path)
kb_tokenizer = KeyboardTokenizerv1()

USE_TIME = False
USE_VELOCITY = True
USE_ACCELERATION = True

In [61]:
state_dict = get_state_dict_from_checkpoint(
    torch.load(CHECKPOINT_PATH, map_location='cpu', weights_only=True))

model = MODEL_GETTERS_DICT[MODEL_NAME]().eval()
model.load_state_dict(state_dict)

/home/proshian/Documents/executorch_test_examples/executorch_examples_venv/lib/python3.12/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


<All keys matched successfully>

In [62]:
def _prepare_encoder_input(encoder_in: Union[Tensor, Tuple[Tensor, Tensor]], 
                           device: str, batch_first: bool
                           ) -> Tuple[Tensor, Tensor]:
    is_tensor = None
    if isinstance(encoder_in, Tensor):
        is_tensor = True
        encoder_in = [encoder_in]
    else:
        is_tensor = False

    encoder_in = [el.unsqueeze(0) if batch_first else el.unsqueeze(1) for el in encoder_in]
    encoder_in = [el.to(device) for el in encoder_in]

    return encoder_in[0] if is_tensor else encoder_in

In [63]:
RAW_DATASET_ITEM_EXAMPLE = (
    array.array('h', [567, 567, 507, 424, 380, 348, 337, 332, 330, 329, 327, 326, 326]),
    array.array('h', [66, 66, 101, 161, 196, 230, 240, 245, 247, 249, 251, 251, 251]),
    array.array('h', [0, 3, 24, 52, 75, 90, 106, 129, 145, 161, 177, 195, 209]),
    'default',
    'на')

GRIDNAME_TO_GRID_PATH = '../data/data_separated_grid/gridname_to_grid.json'

In [64]:
transform = get_val_transform(
    gridname_to_grid_path=GRIDNAME_TO_GRID_PATH,
    grid_names=['default'],
    transform_name=TRANSFORM_NAME,
    char_tokenizer=char_tokenizer,
    uniform_noise_range=0,
    include_time=USE_TIME,
    include_velocities=USE_VELOCITY,
    include_accelerations=USE_ACCELERATION,
    dist_weights_func=None,  # Fill if weighted version is used
    ds_paths_list=[],
)


(encoder_in, decoder_in), decoder_out_target = transform(RAW_DATASET_ITEM_EXAMPLE)
encoder_in = _prepare_encoder_input(encoder_in, 'cpu', batch_first=False)
if isinstance(encoder_in, list):
    encoder_in = tuple(encoder_in)
decoder_in = decoder_in.unsqueeze(1).to(torch.int32)

encoded = model.encode(encoder_in, None)

Accumulating out-of-bounds coordinates...
augmenting gname_to_out_of_bounds


In [65]:
# # Get data manually
# SWIPE_LENGTH = 13
# BATCH_SIZE = 1
# NUM_TRAJ_FEATS = 6
# OUT_SEQ_LEN = 3

# sample_kb_key_ids = torch.ones((SWIPE_LENGTH, BATCH_SIZE), dtype=torch.int32)
# sample_traj_feats = torch.ones((SWIPE_LENGTH, BATCH_SIZE, NUM_TRAJ_FEATS), dtype=torch.float32)
# encoder_in = (sample_traj_feats, sample_kb_key_ids)
# decoder_in = torch.ones((OUT_SEQ_LEN, BATCH_SIZE), dtype=torch.int32)
# encoded = model.encode(
#     encoder_in, 
#     None)

In [ ]:
class Encode(torch.nn.Module):
    def __init__(self, model) -> None:
        super().__init__()
        self.enc_in_emb_model = model.enc_in_emb_model
        self.encoder = model.encoder

    def forward(self, encoder_in):
        x = self.enc_in_emb_model(encoder_in)
        result = self.encoder(x, src_key_padding_mask = None)
        return result


class Decode(torch.nn.Module):
    def __init__(self, model) -> None:
        super().__init__()
        self.dec_in_emb_model = model.dec_in_emb_model
        self.decoder = model.decoder
        self._get_mask = model._get_mask
        self.out = model.out

    def forward(self, decoder_in, x_encoded):
        y = self.dec_in_emb_model(decoder_in)
        tgt_mask = self._get_mask(y.size(0))  # = self.causal_mask[y.size(0):, y.size(0):]
        dec_out = self.decoder(
            y, x_encoded, tgt_mask=tgt_mask, 
            memory_key_padding_mask=None, 
            tgt_key_padding_mask=None,
            tgt_is_causal=True)
        return self.out(dec_out)
    

MAX_SWIPE_LEN = 299
MAX_WORD_LEN = 35
dim_swipe_seq = Dim("dim_swipe_seq", min=1, max=MAX_SWIPE_LEN)
dim_char_seq = Dim("dim_char_seq", min=1, max=MAX_WORD_LEN)

encoder_dynamic_shapes = {"encoder_in": ({0: dim_swipe_seq}, {0: dim_swipe_seq})}
decoder_dynamic_shapes = {
    "x_encoded": {0: dim_swipe_seq},
    "decoder_in": {0: dim_char_seq}
}

aten_encode: ExportedProgram = export(Encode(model).eval(), (encoder_in,), dynamic_shapes=encoder_dynamic_shapes)
aten_decode: ExportedProgram = export(Decode(model).eval(), (decoder_in, encoded), dynamic_shapes=decoder_dynamic_shapes)

edge_xnnpack: EdgeProgramManager = to_edge_transform_and_lower(
    {"encode": aten_encode, "decode": aten_decode},
    partitioner=[XnnpackPartitioner()],
)

exec_prog_xnnpack = edge_xnnpack.to_executorch()

with open("xnnpack_my_nearest_feats.pte", "wb") as file:
    exec_prog_xnnpack.write_to_file(file)

In [ ]:
edge_program = to_edge({"encode": aten_encode, "decode": aten_decode})

executorch_program = edge_program.to_executorch()

with open("raw_my_nearest_feats.pte", "wb") as file:
    file.write(executorch_program.buffer)


In [16]:
# print(aten_encode)

In [17]:
# print(exec_prog_xnnpack.exported_program('encode'))

In [18]:
# exec_prog_xnnpack.exported_program('encode').module()